In [4]:
import os
import cv2
import csv
import math
import scipy
import sklearn
import numpy as np
import pandas as pd
import mediapipe as mp
import tensorflow as tf
import plotly.express as px 
from tqdm.notebook import tqdm
import tensorflow_addons as tfa
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from IPython.display import HTML
from tensorflow.keras import layers
import matplotlib.animation as animation
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GroupShuffleSplit 

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [5]:
# If True, processing data from scratch
# If False, loads preprocessed data
PREPROCESS_DATA = True
TRAIN_MODEL = True
# True: use 10% of participants as validation set
# False: use all data for training -> gives better LB result
USE_VAL = False
N_ROWS = 543
N_DIMS = 3
DIM_NAMES = ['x', 'y', 'z']
SEED = 42
NUM_CLASSES = 250
INPUT_SIZE = 64
BATCH_ALL_SIGNS_N = 3
BATCH_SIZE = 512
# LR_MAX = 1e-3
LR_MAX = 0.005 # #3
# LR_MAX = 0.00155 # #3
N_WARMUP_EPOCHS = 50
WD_RATIO = 0.05
MASK_VAL = 4237
N_EPOCHS = 250
VERBOSE = 1

In [11]:
USE_TYPES = ['left_hand', 'pose', 'right_hand']
LHAND = np.arange(468, 489) # 21
RHAND = np.arange(522, 543) # 21
# POSE  = np.arange(489, 522)# 33
FACE  = np.arange(0,468) #468
LIP = np.array([ 0, 
    61, 185, 40, 39, 37, 267, 269, 270, 409,
    291, 146, 91, 181, 84, 17, 314, 405, 321, 375,
    78, 191, 80, 81, 82, 13, 312, 311, 310, 415,
    95, 88, 178, 87, 14, 317, 402, 318, 324, 308,
])
LPOSE = np.array([502, 504, 506, 508, 510])
RPOSE = np.array([503, 505, 507, 509, 511])


In [12]:
# Concatenate the relevant landmarks for each dominant hand scenario
left_hand_dominant = np.concatenate((LIP, LHAND, LPOSE))
right_hand_dominant = np.concatenate((LIP, RHAND, RPOSE))
# landmarks = np.concatenate((LIP, RHAND, LHAND, POSE))
hand_index = np.concatenate((LHAND, RHAND), axis = 0)

#Landmark Indices in preprocess data
hands_index = np.argwhere(np.isin(left_hand_dominant, hand_index)).squeeze()
lip_index = np.argwhere(np.isin(left_hand_dominant, LIP)).squeeze()
left_hand_index = np.argwhere(np.isin(left_hand_dominant, LHAND)).squeeze()
right_hand_index = np.argwhere(np.isin(left_hand_dominant, RHAND)).squeeze()
pose_index = np.argwhere(np.isin(left_hand_dominant, LPOSE)).squeeze()
n_cols = left_hand_dominant.size

In [13]:
print(f'# HAND_IDXS: {len(hands_index)}, N_COLS: {n_cols}')

# HAND_IDXS: 21, N_COLS: 66


In [14]:
LIPS_START = 0
LEFT_HAND_START = lip_index.size
RIGHT_HAND_START = LEFT_HAND_START + left_hand_index.size
POSE_START = RIGHT_HAND_START + right_hand_index.size
print(f'LIPS_START: {LIPS_START}, LEFT_HAND_START: {LEFT_HAND_START}, RIGHT_HAND_START: {RIGHT_HAND_START}, POSE_START: {POSE_START}')

LIPS_START: 0, LEFT_HAND_START: 40, RIGHT_HAND_START: 61, POSE_START: 61


In [24]:
# BASE_DIR = 'sign_data/asl-signs'
BASE_DIR = '/kaggle/input/asl-signs'
train = pd.read_csv(f'{BASE_DIR}/train.csv')

In [25]:
# Read Training Data
if PREPROCESS_DATA:
    train = pd.read_csv(f'{BASE_DIR}/train.csv').sample(int(40e3), random_state=SEED)
else:
    train = pd.read_csv(f'{BASE_DIR}/train.csv')

N_SAMPLES = len(train)
print(f'N_SAMPLES: {N_SAMPLES}')

N_SAMPLES: 40000


In [26]:
train

,path,participant_id,sequence_id,sign
56533,train_landmark_files/28656/3311214787.parquet,28656,3311214787,sticky
63119,train_landmark_files/53618/3588192588.parquet,53618,3588192588,before
8760,train_landmark_files/4718/1363575346.parquet,4718,1363575346,pretty
93310,train_landmark_files/37779/951199059.parquet,37779,951199059,hen
44842,train_landmark_files/36257/283190141.parquet,36257,283190141,tomorrow
...,...,...,...,...
90202,train_landmark_files/53618/821142908.parquet,53618,821142908,green
88470,train_landmark_files/25571/75047620.parquet,25571,75047620,will
46001,train_landmark_files/36257/2879593392.parquet,36257,2879593392,wake
34340,train_landmark_files/37055/2405753437.parquet,37055,2405753437,glasswindow


In [27]:
# Get complete file path to file
def get_file_path(path):
    return f'/kaggle/input/asl-signs/{path}'

train['file_path'] = train['path'].apply(get_file_path)

In [28]:
train

,path,participant_id,sequence_id,sign,file_path
56533,train_landmark_files/28656/3311214787.parquet,28656,3311214787,sticky,/kaggle/input/asl-signs/train_landmark_files/2...
63119,train_landmark_files/53618/3588192588.parquet,53618,3588192588,before,/kaggle/input/asl-signs/train_landmark_files/5...
8760,train_landmark_files/4718/1363575346.parquet,4718,1363575346,pretty,/kaggle/input/asl-signs/train_landmark_files/4...
93310,train_landmark_files/37779/951199059.parquet,37779,951199059,hen,/kaggle/input/asl-signs/train_landmark_files/3...
44842,train_landmark_files/36257/283190141.parquet,36257,283190141,tomorrow,/kaggle/input/asl-signs/train_landmark_files/3...
...,...,...,...,...,...
90202,train_landmark_files/53618/821142908.parquet,53618,821142908,green,/kaggle/input/asl-signs/train_landmark_files/5...
88470,train_landmark_files/25571/75047620.parquet,25571,75047620,will,/kaggle/input/asl-signs/train_landmark_files/2...
46001,train_landmark_files/36257/2879593392.parquet,36257,2879593392,wake,/kaggle/input/asl-signs/train_landmark_files/3...
34340,train_landmark_files/37055/2405753437.parquet,37055,2405753437,glasswindow,/kaggle/input/asl-signs/train_landmark_files/3...


### Ordinal Encode Sign

In [29]:
import json

with open('/kaggle/input/asl-signs/sign_to_prediction_index_map.json') as f: 
    sign_map = json.load(f)

signs = list(sign_map)


In [30]:
signs[:5]

['TV', 'after', 'airplane', 'all', 'alligator']

In [31]:
def mapping_sign_code(sign: str):
    return sign_map[sign]
train['sign_code'] = train['sign'].apply(mapping_sign_code)

In [32]:
# Instantiate the encoder
le = LabelEncoder()

# Fit the encoder and transform the 'sign' column
train['sign_code'] = le.fit_transform(train['sign'])

# Create dictionaries for mapping
SIGN2ORD = dict(zip(le.classes_, le.transform(le.classes_)))
ORD2SIGN = dict(zip(le.transform(le.classes_), le.classes_))

In [33]:
display(train.head(10))

,path,participant_id,sequence_id,sign,file_path,sign_code
56533,train_landmark_files/28656/3311214787.parquet,28656,3311214787,sticky,/kaggle/input/asl-signs/train_landmark_files/2...,206
63119,train_landmark_files/53618/3588192588.parquet,53618,3588192588,before,/kaggle/input/asl-signs/train_landmark_files/5...,20
8760,train_landmark_files/4718/1363575346.parquet,4718,1363575346,pretty,/kaggle/input/asl-signs/train_landmark_files/4...,178
93310,train_landmark_files/37779/951199059.parquet,37779,951199059,hen,/kaggle/input/asl-signs/train_landmark_files/3...,114
44842,train_landmark_files/36257/283190141.parquet,36257,283190141,tomorrow,/kaggle/input/asl-signs/train_landmark_files/3...,221
20993,train_landmark_files/61333/186594661.parquet,61333,186594661,up,/kaggle/input/asl-signs/train_landmark_files/6...,230
89267,train_landmark_files/53618/782770724.parquet,53618,782770724,blow,/kaggle/input/asl-signs/train_landmark_files/5...,25
48370,train_landmark_files/16069/2977903115.parquet,16069,2977903115,weus,/kaggle/input/asl-signs/train_landmark_files/1...,236
41330,train_landmark_files/2044/269101282.parquet,2044,269101282,read,/kaggle/input/asl-signs/train_landmark_files/2...,184
53090,train_landmark_files/28656/3171133897.parquet,28656,3171133897,say,/kaggle/input/asl-signs/train_landmark_files/2...,191


In [34]:
ROWS_PER_FRAME = 543  

def load_relevant_data_subset(pq_path):
    data_columns = ['x', 'y', 'z']
    data = pd.read_parquet(pq_path, columns=data_columns)
    n_frames = int(len(data) / ROWS_PER_FRAME)
    data = data.values.reshape(n_frames, ROWS_PER_FRAME, len(data_columns))
    return data.astype(np.float32)

In [35]:
def pad_edge(t, repeats, side):
    if side == 'LEFT':
        return tf.concat((tf.repeat(t[:1], repeats=repeats, axis=0), t), axis=0)
    elif side == 'RIGHT':
        return tf.concat((t, tf.repeat(t[-1:], repeats=repeats, axis=0)), axis=0)


In [36]:
"""
Tensorflow layer to process data within TFLite.
To avoid external dependencies, data processing is embedded within the model.
"""
class TFLitePreprocessLayer(tf.keras.layers.Layer):
    def __init__(self):
        super(TFLitePreprocessLayer, self).__init__()

    @tf.function(input_signature=(tf.TensorSpec(shape=[None, N_ROWS, N_DIMS], dtype=tf.float32),))
    def call(self, inputData):

        totalFrames = tf.shape(inputData)[0]

        # Calculate the dominant hand based on sum of absolute coordinates
        leftSum = tf.math.reduce_sum(tf.where(tf.math.is_nan(tf.gather(inputData, LHAND, axis=1)), 0, 1))
        rightSum = tf.math.reduce_sum(tf.where(tf.math.is_nan(tf.gather(inputData, RHAND, axis=1)), 0, 1))
        isLeftDominant = leftSum >= rightSum

        handIndices = LHAND if isLeftDominant else RHAND
        framesWithHand = tf.math.reduce_sum(tf.where(tf.math.is_nan(tf.gather(inputData, handIndices, axis=1)), 0, 1), axis=[1, 2])
        validFrameIndices = tf.squeeze(tf.where(framesWithHand > 0), axis=1)

        processedData = tf.gather(inputData, validFrameIndices, axis=0)
        validFrameIndices = tf.cast(validFrameIndices, tf.float32)
        validFrameIndices -= tf.reduce_min(validFrameIndices)

        frameCount = tf.shape(processedData)[0]

        landmarkColumns = left_hand_dominant if isLeftDominant else right_hand_dominant
        processedData = tf.gather(processedData, landmarkColumns, axis=1)

        # Check if video fits within the input size
        if frameCount < INPUT_SIZE:
            validFrameIndices = tf.pad(validFrameIndices, [[0, INPUT_SIZE - frameCount]], constant_values=-1)
            processedData = tf.pad(processedData, [[0, INPUT_SIZE - frameCount], [0, 0], [0, 0]], constant_values=0)
            processedData = tf.where(tf.math.is_nan(processedData), 0.0, processedData)
            return processedData, validFrameIndices
        else:
            # Handling videos larger than the input size
            if frameCount < INPUT_SIZE ** 2:
                factor = tf.math.floordiv(INPUT_SIZE * INPUT_SIZE, totalFrames)
                processedData = tf.repeat(processedData, repeats=factor, axis=0)
                validFrameIndices = tf.repeat(validFrameIndices, repeats=factor, axis=0)

            adjustedSize = tf.math.floordiv(len(processedData), INPUT_SIZE)
            if tf.math.mod(len(processedData), INPUT_SIZE) > 0:
                adjustedSize += 1

            requiredPadding = (adjustedSize * INPUT_SIZE) - len(processedData) if adjustedSize == 1 else (adjustedSize * INPUT_SIZE) % len(processedData)
            padLeft = tf.math.floordiv(requiredPadding, 2) + tf.math.floordiv(INPUT_SIZE, 2)
            padRight = padLeft + (1 if tf.math.mod(requiredPadding, 2) > 0 else 0)

            processedData = self._addPadding(processedData, padLeft, 'LEFT')
            processedData = self._addPadding(processedData, padRight, 'RIGHT')
            validFrameIndices = self._addPadding(validFrameIndices, padLeft, 'LEFT')
            validFrameIndices = self._addPadding(validFrameIndices, padRight, 'RIGHT')

            processedData = tf.reshape(processedData, [INPUT_SIZE, -1, n_cols, N_DIMS])
            validFrameIndices = tf.reshape(validFrameIndices, [INPUT_SIZE, -1])
            
            processedData = tf.experimental.numpy.nanmean(processedData, axis=1)
            validFrameIndices = tf.experimental.numpy.nanmean(validFrameIndices, axis=1)

            processedData = tf.where(tf.math.is_nan(processedData), 0.0, processedData)
            return processedData, validFrameIndices

    def _addPadding(self, tensor, paddingSize, direction):
        if direction == 'LEFT':
            return tf.concat([tf.repeat(tensor[:1], repeats=paddingSize, axis=0), tensor], axis=0)
        else:
            return tf.concat([tensor, tf.repeat(tensor[-1:], repeats=paddingSize, axis=0)], axis=0)

layerInstance = TFLitePreprocessLayer()


In [37]:
def get_data(file_path): 
    # Load Raw Data
    data_2 = load_relevant_data_subset(file_path)
    # Process Data Using Tensorflow
    data_2 = layerInstance(data_2)
    return data_2

In [38]:
# def create_dataset(N_SAMPLES, INPUT_SIZE, N_COLS, N_DIMS, file_paths, sign_ords, participant_ids, SEED=42):
def create_dataset():
    # Create arrays to save data
    X = np.zeros([N_SAMPLES,  INPUT_SIZE, n_cols, N_DIMS], dtype=np.float32)
    y = np.zeros([N_SAMPLES ], dtype=np.int32)
    NON_EMPTY_FRAME_IDXS = np.full([N_SAMPLES, INPUT_SIZE], -1, dtype=np.float32)

    # Fill X/y
    for row_idx, (file_path, sign_code) in enumerate(tqdm(train[['file_path', 'sign_code']].itertuples(index=False))):
        # Log message every 5000 samples
        if row_idx == N_SAMPLES: 
            break
        data, non_empty_frame_idxs = get_data(file_path)
        X[row_idx] = data
        y[row_idx] = sign_code
        NON_EMPTY_FRAME_IDXS[row_idx] = non_empty_frame_idxs
        # Sanity check, data should not contain NaN values
        if np.isnan(data).sum() > 0:
            print(row_idx)
            return data

    # Save X/y
    np.save('X.npy', X)
    np.save('y.npy', y)
    np.save('NON_EMPTY_FRAME_IDXS.npy', NON_EMPTY_FRAME_IDXS)

    # Use GroupShuffleSplit to get train and a temporary set (temp_set)
    splitter_1 = GroupShuffleSplit(test_size=0.20, n_splits=1, random_state=SEED)  # Assuming 20% is for validation+test
    participant_ids = train['participant_id'].values
    train_idxs, temp_idxs = next(splitter_1.split(X, y, groups=participant_ids))

    # Further split temp_set into validation and test sets
    splitter_2 = GroupShuffleSplit(test_size=0.50, n_splits=1, random_state=SEED)  # Splitting temp_set into half for val and test
    X_temp, y_temp, participant_ids_temp = X[temp_idxs], y[temp_idxs], participant_ids[temp_idxs]
    val_idxs_temp, test_idxs_temp = next(splitter_2.split(X_temp, y_temp, groups=participant_ids_temp))

    # Convert relative val and test indices to original indices
    val_idxs = temp_idxs[val_idxs_temp]
    test_idxs = temp_idxs[test_idxs_temp]
    
    # Save Train
    X_train = X[train_idxs]
    NON_EMPTY_FRAME_IDXS_TRAIN = NON_EMPTY_FRAME_IDXS[train_idxs]
    y_train = y[train_idxs]
    np.save('X_train.npy', X_train)
    np.save('y_train.npy', y_train)
    np.save('NON_EMPTY_FRAME_IDXS_TRAIN.npy', NON_EMPTY_FRAME_IDXS_TRAIN)

    # Save Validation
    X_val = X[val_idxs]
    NON_EMPTY_FRAME_IDXS_VAL = NON_EMPTY_FRAME_IDXS[val_idxs]
    y_val = y[val_idxs]
    np.save('X_val.npy', X_val)
    np.save('y_val.npy', y_val)
    np.save('NON_EMPTY_FRAME_IDXS_VAL.npy', NON_EMPTY_FRAME_IDXS_VAL)

    # Save Test
    X_test = X[test_idxs]
    NON_EMPTY_FRAME_IDXS_TEST = NON_EMPTY_FRAME_IDXS[test_idxs]
    y_test = y[test_idxs]
    np.save('X_test.npy', X_test)
    np.save('y_test.npy', y_test)
    np.save('NON_EMPTY_FRAME_IDXS_TEST.npy', NON_EMPTY_FRAME_IDXS_TEST)
    
    # Split Statistics
    print(f'Patient ID Intersection Train/Val: {set(participant_ids[train_idxs]).intersection(participant_ids[val_idxs])}')
    print(f'X_train shape: {X_train.shape}, X_test shape: {X_test.shape}, X_val shape: {X_val.shape}')
    print(f'y_train shape: {y_train.shape}, y_test shape: {y_test.shape}, y_val shape: {y_val.shape}')

create_dataset()

0it [00:00, ?it/s]

Patient ID Intersection Train/Val: set()
X_train shape: (30550, 64, 66, 3), X_test shape: (5429, 64, 66, 3), X_val shape: (4021, 64, 66, 3)
y_train shape: (30550,), y_test shape: (5429,), y_val shape: (4021,)
